In [7]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from pyvis.network import Network as net

In [12]:
g = nx.read_gpickle("objects/graph-A_3_0")

In [13]:
print(nx.info(g))

DiGraph named 'Graph from soil food web adjacency matrix' with 24 nodes and 94 edges


In [14]:
g.nodes(data=True)

NodeDataView({'11 Plant-feeding nematode': {'trophic_level': 1, 'Biomass': 479278.83999999997, 'Mass': 6.031837019417812}, '31 Bacterivore nematode': {'trophic_level': 1, 'Biomass': 97189.95, 'Mass': 0.44778320101132774}, '45 Substrate-ingesting earthworm': {'trophic_level': 1, 'Biomass': 20876194.67, 'Mass': 656677.5800017535}, '23 Fungivore insect': {'trophic_level': 1, 'Biomass': 87314.16, 'Mass': 123.42029317311494}, '24 Fungivore enchytraeid': {'trophic_level': 1, 'Biomass': 666025.6699999999, 'Mass': 469.51500281899325}, '21 Fungivore nematode': {'trophic_level': 1, 'Biomass': 17869.440000000002, 'Mass': 0.060594173232088044}, '81 Omnivore nematode': {'trophic_level': 2, 'Biomass': 301148.6, 'Mass': 1.6196323444430343}, '13 Plant-feeding insect (springtail)': {'trophic_level': 1, 'Biomass': 12483.4, 'Mass': 8.82985650742963}, '44 Substrate-inhabiting enchytraeid': {'trophic_level': 1, 'Biomass': 213718.46000000002, 'Mass': 109.48778868944353}, '51 Predating nematode (consuming ne

### Create network divided by trophic level 0 as source

In [188]:
# list of primary producers
prim_prod = ["#0 Plant autotrophs (roots)", "48 Primary (heterotrophic) producer bacteria", "49 Primary (heterotrophic) producer fungi"]

#### #0 Plant autotrophs (roots)

In [189]:
# Keep only one primary producer in a new network
g0 = nx.read_gpickle("objects/graph-A_3_0")

# selected primary porducer
primary = "#0 Plant autotrophs (roots)"

# remove other primary producer nodes
for n in prim_prod:
    if n != primary:
        # remove from Nxgraph
        g0.remove_node(n) 

In [190]:
# Remove invalid edges (trophic level must be eaqual or increase from source to target node) 
g_itr = g0.copy()
for source, target in g_itr.edges():
    if g_itr.nodes[source]['trophic_level'] > g_itr.nodes[target]['trophic_level']:
        g0.remove_edge(source, target)

In [191]:
# Remove unreachable nodes

# Find reachable nodes
reach_nodes = nx.descendants(g0, "#0 Plant autotrophs (roots)")
print(reach_nodes)

# unreachable nodes
unreach_nodes = [node for node in list(g0.nodes()) if node not in reach_nodes]
print(unreach_nodes)

# Remove unreachable nodes 
for node in unreach_nodes:
    if node != primary:
        g0.remove_node(node)

{'81 Omnivore nematode', '51 Predating nematode (consuming nematodes)', '13 Plant-feeding insect (springtail)', '45 Substrate-ingesting earthworm', '12 Macrophytophage and panphytophage mite', '11 Plant-feeding nematode', '62 Predatory mite (attacking arthropods)', '82 Omnivore mite', '72 Generalist mite', '92 Predatory mite (parasitizing mites and nematodes)', '52 Predatory mite (attacking nematodes)'}
['31 Bacterivore nematode', '23 Fungivore insect', '24 Fungivore enchytraeid', '21 Fungivore nematode', '44 Substrate-inhabiting enchytraeid', '34 Bacterivore enchytraeid', '36 Bacterivore amoebae', '37 Bacterivore flagellates', '#0 Plant autotrophs (roots)', '41 "Passive lifestage, substrate-related nematode"', '22 Microphytophage mite (feeding on fungi)']


In [192]:
nx.info(g0)

"DiGraph named 'Graph from soil food web adjacency matrix' with 12 nodes and 43 edges"

In [193]:
# Import trophic level as node attribute
trophic_levels = pd.read_csv("data/trophic_levels_1_0.csv", index_col=0, header=None, delimiter=";", names=["trophic_level"])
trophic_levels = trophic_levels.squeeze()

# Create dict from trophic level Series
troph_lev_dict = trophic_levels.to_dict()
troph_lev_dict

{'11 Plant-feeding nematode': 1,
 '31 Bacterivore nematode': 1,
 '45 Substrate-ingesting earthworm': 1,
 '23 Fungivore insect': 1,
 '24 Fungivore enchytraeid': 1,
 '21 Fungivore nematode': 1,
 '81 Omnivore nematode': 2,
 '13 Plant-feeding insect (springtail)': 1,
 '44 Substrate-inhabiting enchytraeid': 1,
 '51 Predating nematode (consuming nematodes)': 3,
 '72 Generalist mite': 3,
 '34 Bacterivore enchytraeid': 1,
 '82 Omnivore mite': 2,
 '12 Macrophytophage and panphytophage mite': 1,
 '36 Bacterivore amoebae': 1,
 '37 Bacterivore flagellates': 1,
 '#0 Plant autotrophs (roots)': 0,
 '41 "Passive lifestage, substrate-related nematode"': 1,
 '48 Primary (heterotrophic) producer bacteria': 0,
 '49 Primary (heterotrophic) producer fungi': 0,
 '22 Microphytophage mite (feeding on fungi)': 1,
 '62 Predatory mite (attacking arthropods)': 3,
 '52 Predatory mite (attacking nematodes)': 3,
 '92 Predatory mite (parasitizing mites and nematodes)': 3}

In [194]:
tlevel = list(troph_lev_dict.values())
colours = ["#40E0D0", "#DFFF00", "#FFBF00", "#DE3163"]
tlevel2color = {lev: colours[lev] for lev in tlevel}

In [195]:
# # create nodes list
# nodes0 = list(g0.nodes())

In [196]:
def chunkstring(string, length):
    return (string[0+i:length+i] for i in range(0, len(string), length))

# Create a PyVis graph G
G0 = net('800px', '1200px')
# nodes = list(g.nodes())
for node in g0.nodes():
    G0.add_node(node, 
               label='\n'.join(chunkstring(node,15)), 
               level=max(trophic_levels) - troph_lev_dict[node], 
               shape='box',
               color=tlevel2color[troph_lev_dict[node]])

In [197]:
# Add edges to PyVis graph
for source, target in list(g0.edges()):
    G0.add_edge(source, target)

In [198]:
# Edges take on the color of the node they are coming from.
G0.inherit_edge_colors(status=True)

In [202]:
options= """
var options = {
  "edges": {
    "arrows": {
      "middle": {
        "enabled": true,
        "scaleFactor": 0.85
      }
    },
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "layout": {
    "hierarchical": {
      "enabled": true,
      "nodeSpacing": 200
    }
  },
  "interaction": {
    "hover": true,
    "navigationButtons": true,
    "multiselect": true
  },
  "manipulation": {
    "enabled": true,
    "initiallyActive": true
  },
  "physics": {
    "enabled": true,
    "hierarchicalRepulsion": {
      "centralGravity": 0,
      "nodeDistance": 190
    },
    "minVelocity": 0.75,
    "solver": "hierarchicalRepulsion"
  },
  "configure": {
    "enabled": true
  }
}
"""
# G.write_html('SFW_2.html')
G0.conf = True
G0.set_options(options)
# G.show_buttons(filter_=True)  # if we enable this after setting options there is a bug so we use conf=True and JS settings
G0.write_html('SFW0-0_1.html')

In [199]:
# G0.show_buttons(filter_=True)
# G0.show('SFW0-0_1.html')

In [ ]:
# G48.set_options("""
# var options = {
#   "nodes": {
#     "font": {
#       "size": 15,
#       "strokeWidth": 5
#     }
#   },
#   "edges": {
#     "arrows": {
#       "middle": {
#         "enabled": true,
#         "scaleFactor": 0.8
#       }
#     },
#     "color": {
#       "inherit": true
#     },
#     "smooth": false
#   },
#   "physics": {
#     "enabled": false,
#     "minVelocity": 0.75
#   }
# }
# """) 
# G48.show('SFW-48_1.html')